<a href="https://colab.research.google.com/github/wooihaw/ERA3036_T2310/blob/main/Tutorial_3/Tutorial_3_Coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initialization
%matplotlib inline
from warnings import filterwarnings
filterwarnings('ignore')

## Exercise 1
### Regression
#### Build a regression model to estimate the weight based on height

In [ ]:
import sys
from pandas import read_csv, get_dummies
from sklearn.model_selection import train_test_split as split
from sklearn.linear_model import LinearRegression

# Determine the environment
is_colab = 'google.colab' in sys.modules
# # Load the dataset conditionally
if is_colab:
    # Code for Google Colab environment
    df = read_csv("https://raw.githubusercontent.com/wooihaw/ERA3036_T2310/main/data/genders_heights_weights.csv")
else:
    # Code for local Jupyter Notebook environment
    df = read_csv("../data/genders_heights_weights.csv")

X1 = df.drop(columns=['Gender', 'Weight'])
y1 = df['Weight']
X1_train, X1_test, y1_train, y1_test = split(X1, y1, train_size=0.75, random_state=42)

To do:
- Build a linear regression model (name it as lnr1) to estimate weight using height
- Evaluate the model's performance with R2 score

In [ ]:
h = eval(input('Enter height: '))
print(f'Estimated weight is: {lnr1.predict([[h]])}kg')

To do:
- Apply categorical encoding using the get_dummies() function from pandas
- Print 5 random data samples

To do:
- Separate into fetures and targets
- Split to training and testing sets

To do:
- Build another linear regression model to estimate the weight based on height and both genders

In [ ]:
h = eval(input('Enter height: '))
g = eval(input('Enter gender (0 for male, 1 for female): '))
print(f'Estimated weight is: {lnr2.predict([[h, g, 1-g]])}kg')

## Exercise 2
### Classification
#### Compare 7 classification models for handwritten digit recognition

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split as split, KFold, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.decomposition import PCA

X, y = load_digits(return_X_y=True)

# View one of the image
plt.imshow(X[0, :].reshape(8, 8), cmap='gray')
plt.axis(False)
plt.title(f'Digit: {y[0]}')
plt.show()

To do:
- Check the number of features in the dataset

To do:
- Use spot-checking technique to compare 7 classification models

https://machinelearningmastery.com/spot-check-machine-learning-algorithms-in-python/
1. Load Dataset

In [ ]:
# load the dataset, returns X and y elements
def load_dataset():
    X, y = None, None
    return X, y

2. Define Models

In [ ]:
# create a dict of standard models to evaluate {name:object}
def define_models(models=dict()):
    # ...
    return models

3. Evaluate Models

In [ ]:
# create a feature preparation pipeline for a model
def make_pipeline(model):
    steps = list()
    # standardization
    steps.append(('standardize', StandardScaler()))
    # normalization
    steps.append(('normalize', MinMaxScaler()))
    # the model
    steps.append(('model', model))
    # create pipeline
    pipeline = Pipeline(steps=steps)
    return pipeline

Evaluate a single model

In [ ]:
# evaluate a single model
def evaluate_model(X, y, model, folds, metric):
    # create the pipeline
    pipeline = make_pipeline(model)
    # evaluate model
    scores = cross_val_score(pipeline, X, y, scoring=metric, cv=folds, n_jobs=-1)
    return scores

In [ ]:
# evaluate a model and try to trap errors and and hide warnings
def robust_evaluate_model(X, y, model, folds, metric):
    scores = None
    try:
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            scores = evaluate_model(X, y, model, folds, metric)
    except:
        scores = None
    return scores

Evaluate a dictionary of models

In [ ]:
# evaluate a dict of models {name:object}, returns {name:score}
def evaluate_models(X, y, models, folds=10, metric='accuracy'):
    results = dict()
    for name, model in models.items():
        # evaluate the model
        scores = robust_evaluate_model(X, y, model, folds, metric)
        # show process
        if scores is not None:
            # store a result
            results[name] = scores
            mean_score, std_score = mean(scores), std(scores)
            print('>%s: %.3f (+/-%.3f)' % (name, mean_score, std_score))
        else:
            print('>%s: error' % name)
    return results

Summarize Results

In [ ]:
# print and plot the top n results
def summarize_results(results, maximize=True, top_n=10):
    # check for no results
    if len(results) == 0:
        print('no results')
        return
    # determine how many results to summarize
    n = min(top_n, len(results))
    # create a list of (name, mean(scores)) tuples
    mean_scores = [(k,mean(v)) for k,v in results.items()]
    # sort tuples by mean score
    mean_scores = sorted(mean_scores, key=lambda x: x[1])
    # reverse for descending order (e.g. for accuracy)
    if maximize:
        mean_scores = list(reversed(mean_scores))
    # retrieve the top n for summarization
    names = [x[0] for x in mean_scores[:n]]
    scores = [results[x[0]] for x in mean_scores[:n]]
    # print the top n
    print()
    for i in range(n):
        name = names[i]
        mean_score, std_score = mean(results[name]), std(results[name])
        print('Rank=%d, Name=%s, Score=%.3f (+/- %.3f)' % (i+1, name, mean_score, std_score))
    # boxplot for the top n
    pyplot.boxplot(scores, labels=names)
    _, labels = pyplot.xticks()
    pyplot.setp(labels, rotation=90)
    pyplot.savefig('spotcheck.png')

To do:
- Use Univariate Selection to select 20 best features. Evaluate the performance of the best model above on these features using 5-fold cross validation.